In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns
plt.rcParams["figure.figsize"]=[15,5]

In [2]:
raw_sp = pd.read_csv('screenprinter_plus_vaf.csv', usecols =['timestamp', 'data.A1'])
raw_sp.dropna(axis=0, inplace=True)
raw_sp.sort_values(by=['timestamp'])
raw_sp.reset_index(inplace=True)
raw_sp['timestamp'] =  pd.to_datetime(raw_sp['timestamp'])

In [3]:
test_sp=raw_sp
sptimethresh=12

In [4]:
#Get boards from Screenprinter
test_sp=test_sp[['data.A1','timestamp']]
# events=scipy.signal.find_peaks(test_sp['data.A1'], height=(2.22, 5), width=1)

In [5]:
test_sp['sum']=pd.Series.to_frame(test_sp['data.A1'].rolling(6, center=True).sum())
test_sp['sum_forcleaning']=pd.Series.to_frame(test_sp['data.A1'].rolling(12, center=True).sum())

In [6]:
%matplotlib qt

In [9]:
plt.ylabel="Screenprinter Current (A)"
plt.plot(test_sp['timestamp'], test_sp['data.A1'])


In [58]:
plt.plot(test_sp['timestamp'], test_sp['sum'])
plt.plot(test_sp['timestamp'], test_sp['data.A1']*10)

In [17]:
plt.plot(test_sp['timestamp'], test_sp['sum'])

In [59]:
test_sp['idle']=2

In [36]:
sptimethresh=5
printing_delays_raw=scipy.signal.find_peaks(test_sp['sum'], height=(11, 15), distance=sptimethresh, width=1)
printing_delays_raw_df=pd.DataFrame({"sample_number":printing_delays_raw[0], "working_time":printing_delays_raw[1]['widths']})
cleaning_delays_raw=scipy.signal.find_peaks(test_sp['sum_forcleaning'], height=60, distance=sptimethresh, width=1)
cleaning_delays_raw_df=pd.DataFrame({"sample_number":cleaning_delays_raw[0], "working_time":cleaning_delays_raw[1]['widths']})

In [60]:
for index, i in enumerate(printing_delays_raw[0]):
    test_sp.ix[int(printing_delays_raw[1]['left_ips'][index]) : int(printing_delays_raw[1]['right_ips'][index]), 'idle']=0

for index, i in enumerate(cleaning_delays_raw[0]):
    test_sp.ix[int(cleaning_delays_raw[1]['left_ips'][index]) : int(cleaning_delays_raw[1]['right_ips'][index]), 'idle']=0

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [61]:
idle_delays_raw=scipy.signal.find_peaks(test_sp['idle'], height=.5, width=1)
idle_delays_raw_df=pd.DataFrame({"sample_number":idle_delays_raw[0], "working_time":idle_delays_raw[1]['widths']})

In [62]:
SP_events=pd.DataFrame({"timestamp":test_sp.iloc[printing_delays_raw_df.sample_number].timestamp, "event":1, "working_time":printing_delays_raw_df.working_time.tolist()})
cleanings=pd.DataFrame({"timestamp":test_sp.iloc[cleaning_delays_raw_df.sample_number].timestamp, "event":2, "working_time":cleaning_delays_raw_df.working_time.tolist()})

SP_events.reset_index(inplace=True)
cleanings.reset_index(inplace=True)

In [63]:
idles=pd.DataFrame({"timestamp":test_sp.iloc[idle_delays_raw_df.sample_number].timestamp, "event":0, "working_time":idle_delays_raw_df.working_time.tolist()})
idles.reset_index(inplace=True)

In [64]:
for x, row in SP_events.iterrows():
    SP_events.ix[x,'energy']=test_sp.ix[int(printing_delays_raw[1]['left_ips'][x]):int(printing_delays_raw[1]['right_ips'][x]), 'data.A1'].sum()*230/3600000
# for index, x in enumerate(printing_delays_raw[0]):
#     SP_events.ix[index,'energy']=np.trapz(test_sp.ix[int(printing_delays_raw[1]['left_ips'][index]):int(printing_delays_raw[1]['right_ips'][index]), 'data.A1'].tolist()*230/3600000)

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [65]:
for x, row in cleanings.iterrows():
    cleanings.ix[x,'energy']=test_sp.ix[int(cleaning_delays_raw[1]['left_ips'][x]):int(cleaning_delays_raw[1]['right_ips'][x]), 'data.A1'].sum()*230/3600000

# for index,x in enumerate(cleaning_delays_raw[0]):
#     cleanings.ix[index,'energy']=np.trapz(test_sp.ix[int(cleaning_delays_raw[1]['left_ips'][index]):int(cleaning_delays_raw[1]['right_ips'][index]), 'data.A1'].tolist()*230)
    

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [66]:
for x, row in idles.iterrows():
    idles.ix[x,'energy']=test_sp.ix[int(idle_delays_raw[1]['left_ips'][x]):int(idle_delays_raw[1]['right_ips'][x]), 'data.A1'].sum()*230/3600000

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [67]:
SP_events.index=SP_events.timestamp
SP_events.drop('timestamp', axis=1, inplace=True)
SP_events.drop('index', axis=1, inplace=True)

cleanings.index=cleanings.timestamp
cleanings.drop('timestamp', axis=1, inplace=True)
cleanings.drop('index', axis=1, inplace=True)

idles.index=idles.timestamp
idles.drop('timestamp', axis=1, inplace=True)
idles.drop('index', axis=1, inplace=True)


SP_events=SP_events.append(cleanings)
SP_events=SP_events.append(idles)

In [68]:
# sns.distplot(SP_events.working_time[SP_events['event']==1])
# plt.title('SP Printing Times Histogram')

In [69]:
# sns.distplot(SP_events.working_time[SP_events['event']==2])
# plt.title('SP Cleaning Times Histogram')

In [70]:
print('Printing time Mode: ', SP_events.working_time[SP_events['event']==1].mode().mean())

Printing time Mode:  7.531470655934635


In [71]:
print('Cleaning time Mode: ', SP_events.working_time[SP_events['event']==2].mode().mean())

Cleaning time Mode:  12.308590813828767


In [72]:
SP_events[SP_events.event==1].shape[0]

timestamp
2019-01-22 03:15:24.550434    0.029236
2019-01-22 03:21:15.915506    0.023951
2019-01-22 03:24:04.986968    0.000630
2019-01-22 03:24:56.420928    0.006425
2019-01-22 03:25:50.309151    0.000988
2019-01-22 03:26:30.053417    0.002869
2019-01-22 03:33:48.468700    0.064266
2019-01-22 03:40:54.388389    0.001115
2019-01-22 03:41:31.340452    0.002491
2019-01-22 03:42:49.604958    0.008757
2019-01-22 03:44:04.932283    0.001567
2019-01-22 03:46:46.612632    0.021202
2019-01-22 03:49:36.195888    0.004005
2019-01-22 03:50:21.027965    0.001463
2019-01-22 03:50:58.123721    0.001982
2019-01-22 03:51:36.404514    0.003055
2019-01-22 03:52:13.365261    0.001576
2019-01-22 03:52:46.404299    0.001162
2019-01-22 03:55:10.627585    0.020782
2019-01-22 03:57:35.892162    0.001330
2019-01-22 04:02:14.726063    0.040018
2019-01-22 04:07:03.395226    0.003191
2019-01-22 04:07:44.266206    0.001365
2019-01-22 04:08:16.051696    0.001260
2019-01-22 04:11:59.811775    0.033000
2019-01-22 04:1

In [73]:
cleaning_delays_raw[0][0]

869

In [74]:
# plt.plot(test_sp.timestamp, test_sp.detection)
# plt.plot(test_sp.timestamp, test_sp['sum'])

In [88]:
SP1spower=SP_events[SP_events.event==1]['energy']*3600000/SP_events[SP_events.event==1]['working_time']
SP1spower.mean()

520.0082511685098

In [87]:
SP0spower=SP_events[SP_events.event==0]['energy']*3600000/SP_events[SP_events.event==0]['working_time']
SP0spower.mean()

445.8164916861912

In [56]:
SP_events[SP_events.event==1].shape[0]

191

In [55]:
SP_events[SP_events.event==2].shape[0]

85